In [1]:
#importing all the required libraries
import selenium
from selenium import webdriver
import time
import requests
import json
from bs4 import BeautifulSoup
import os
from PIL import Image
import io
import hashlib
import pandas as pd
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException,ElementNotInteractableException,InvalidArgumentException,InvalidCookieDomainException

### 1. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [34]:
#connecting to the webdriver
driver=webdriver.Chrome("chromedriver.exe")

In [35]:
#specifying the url of the webpage to be scraped
url="https://www.amazon.in/"

In [36]:
#opening the webpage through our driver
driver.get(url)

In [37]:
# Taking input from user 
search = input("Input the string you want to search for:")

#generate a url from search string
url= driver.get("https://www.amazon.in/s?k=" + search + "&start=")

Input the string you want to search for:laptop


### 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [38]:
#getting the list of url of Laptops
Lurl=[]

for j in range(0,3):
    laptop=driver.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']")
    for i in laptop:
        laptop1=i.find_element_by_tag_name("a")
        Lurl.append(laptop1.get_attribute('href'))
try:        
    driver.find_element_by_xpath('//*[@id="search"]/div[1]/div[2]/div/span[3]/div[2]/div[27]/span/div/div/ul/li[3]/a').click()
except NoSuchElementException:
    pass

In [39]:
Lurl


['https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A04778293872IIODJE100&url=%2FHP-Pavilion-14-inch-Windows-14-dv0543TU%2Fdp%2FB09DGC6C67%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dlaptop%26qid%3D1632595497%26sr%3D8-1-spons%26psc%3D1&qualifier=1632595497&id=2124527872900450&widgetName=sp_atf',
 'https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A06203261H6YRAFTM78UJ&url=%2FHP-Chromebook-14a-na0003TU-Touchscreen-Expandable%2Fdp%2FB08C5W6PL2%2Fref%3Dsr_1_2_sspa%3Fdchild%3D1%26keywords%3Dlaptop%26qid%3D1632595497%26sr%3D8-2-spons%26psc%3D1&qualifier=1632595497&id=2124527872900450&widgetName=sp_atf',
 'https://www.amazon.in/Acer-Aspire-Athlon-Dual-core-Processor/dp/B09F9Q17XT/ref=sr_1_3?dchild=1&keywords=laptop&qid=1632595497&qsid=258-7594019-3530040&sr=8-3&sres=B09F9Q17XT%2CB08X4N3DW2%2CB08C5W6PL2%2CB08HJYFZ14%2CB097RKZYGS%2CB0924PSV78%2CB093VZC5X9%2CB08JCP96XL%2CB095JY5B3F%2CB0999BTTKZ%2CB08

In [40]:
#checking the length of tv url
len(Lurl)

78

In [41]:
#going to the url of first television through our driver
driver.get(Lurl[0])

In [42]:
#creating the empty lists
Brandname=[]
Nameofproduct=[]
Rating=[]
Noofrating=[]
Price=[]
Returnexchange=[]
Expecteddelivery=[]
Availibility=[]
Otherdetails=[]

In [43]:
#running a loop to extract all the required information from TV_URL

from tqdm import tqdm
for i in tqdm(Lurl):
    driver.get(i)
    try:
        br=driver.find_element_by_xpath('//*[@id="bylineInfo"]').text
        Brandname.append(br)
    except NoSuchElementException:
        Brandname.append('-')
        
    try:
        pro=driver.find_element_by_xpath('//*[@id="productTitle"]').text
        Nameofproduct.append(pro)
    except NoSuchElementException:
        Nameofproduct.append('-')
    
    try:
        rate_=driver.find_element_by_xpath("//span[@id='acrCustomerReviewText']").click()
        rate=driver.find_element_by_xpath("//span[@class='a-size-medium a-color-base']").text
        Rating.append(rate)
    except NoSuchElementException as e:
        Rating.append('-')
        
    try:
        rating=driver.find_element_by_xpath('//*[@id="acrCustomerReviewText"]').text
        Noofrating.append(rating)
    except NoSuchElementException:
        Noofrating.append('-')
        
    try:
        pri=driver.find_element_by_xpath('//*[@id="priceblock_ourprice"]').text
        Price.append(pri)
    except NoSuchElementException:
        Price.append('-')
        
    try:
        exchange=driver.find_element_by_xpath('//*[@id="icon-farm-container"]/div/div[1]/span/div[2]/a').text
        Returnexchange.append(exchange)
    except NoSuchElementException:
        Returnexchange.append('-')
        
    try:
        delivery=driver.find_element_by_xpath('//*[@id="ddmDeliveryMessage"]/b').text
        Expecteddelivery.append(delivery)
    except NoSuchElementException:
        Expecteddelivery.append('-')
        
    try:
        availibility=driver.find_element_by_xpath('//*[@id="availability"]/span').text
        Availibility.append(availibility)
    except NoSuchElementException:
        Availibility.append('-')
        
    try:
        details=driver.find_element_by_xpath('//*[@id="feature-bullets"]/ul').text.replace("\n"," ")
        Otherdetails.append(details)
    except NoSuchElementException:
        Otherdetails.append('-')
        
    time.sleep(3)

100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [16:04<00:00, 12.37s/it]


In [44]:
Brandname

['Visit the HP Store',
 'Visit the HP Store',
 'Visit the Acer Store',
 'Visit the Dell Store',
 'Visit the HP Store',
 'Visit the Lenovo Store',
 'Visit the Apple Store',
 'Brand: Microsoft',
 'Visit the Acer Store',
 'Visit the HP Store',
 'Visit the ASUS Store',
 'Visit the RDP Store',
 'Visit the HP Store',
 'Visit the Lenovo Store',
 'Visit the LG Store',
 'Visit the Dell Store',
 'Visit the Lenovo Store',
 'Visit the Lenovo Store',
 'Visit the AVITA Store',
 'Visit the ASUS Store',
 'Visit the Lenovo Store',
 'Visit the Lenovo Store',
 'Visit the Acer Store',
 'Visit the Amazon Renewed Store',
 'Visit the Lenovo Store',
 'Brand: MI',
 'Visit the HP Store',
 'Visit the HP Store',
 'Visit the Acer Store',
 'Visit the Dell Store',
 'Visit the HP Store',
 'Visit the Lenovo Store',
 'Visit the Apple Store',
 'Brand: Microsoft',
 'Visit the Acer Store',
 'Visit the HP Store',
 'Visit the ASUS Store',
 'Visit the RDP Store',
 'Visit the HP Store',
 'Visit the Lenovo Store',
 'Visit the 

In [45]:
Nameofproduct

['HP Pavilion 14 11th Gen Intel Core i5 14-inch (35.5 cms) Thin & Light FHD Laptop (8GB/512GB SSD/Windows 10/MS Office/Natural Silver/1.41 kg), 14-dv0543TU',
 'HP Chromebook 14-inch (35.56 cms) Thin & Light Touchscreen Laptop (Celeron N4020/4GB/64GB eMMC + 256GB Expandable Storage/Chrome OS/1.46 kgs Light/ Mineral Silver), 14a-na0003TU',
 'Acer Aspire 3 AMD Athlon Silver 3050U Dual-core Processor 15.6" (39.6 cms) Laptop - (4 GB/1TB HDD/Windows 10 Home/AMD Radeon Graphics /1.9Kg/Silver) A315-23',
 'Dell Inspiron 3502 15.6" (39.62 cms) HD Display Laptop (Pentium Silver N5030 / 4GB / 256GB SSD / Integrated Graphics / Win 10 + MSO / Accent Black) D560427WIN9BE',
 'HP Chromebook 14-inch (35.56 cms) Thin & Light Touchscreen Laptop (Celeron N4020/4GB/64GB eMMC + 256GB Expandable Storage/Chrome OS/1.46 kgs Light/ Mineral Silver), 14a-na0003TU',
 'Lenovo Yoga C940 Intel Core i7 10th Gen 14 inch UHD Thin and Light Laptop (16GB/1TB SSD/Windows 10/Office 2019/Mica/1.25Kg), 81Q9009XIN',
 '2020 Appl

In [46]:
Rating

['4.4 out of 5',
 '3.8 out of 5',
 '3.5 out of 5',
 '4.2 out of 5',
 '3.8 out of 5',
 '4 out of 5',
 '4.7 out of 5',
 '4.1 out of 5',
 '4.1 out of 5',
 '3.5 out of 5',
 '3.8 out of 5',
 '3.3 out of 5',
 '4.3 out of 5',
 '3.3 out of 5',
 '4.5 out of 5',
 '3.8 out of 5',
 '3.4 out of 5',
 '4 out of 5',
 '4 out of 5',
 '4.2 out of 5',
 '3.6 out of 5',
 '4.3 out of 5',
 '4.3 out of 5',
 '5 out of 5',
 '3.8 out of 5',
 '4.2 out of 5',
 '4.4 out of 5',
 '3.8 out of 5',
 '3.5 out of 5',
 '4.2 out of 5',
 '3.8 out of 5',
 '4 out of 5',
 '4.7 out of 5',
 '4.1 out of 5',
 '4.1 out of 5',
 '3.5 out of 5',
 '3.8 out of 5',
 '3.3 out of 5',
 '4.3 out of 5',
 '3.3 out of 5',
 '4.5 out of 5',
 '3.8 out of 5',
 '3.4 out of 5',
 '4 out of 5',
 '4 out of 5',
 '4.2 out of 5',
 '3.6 out of 5',
 '4.3 out of 5',
 '4.3 out of 5',
 '5 out of 5',
 '3.8 out of 5',
 '4.2 out of 5',
 '4.4 out of 5',
 '3.8 out of 5',
 '3.5 out of 5',
 '4.2 out of 5',
 '3.8 out of 5',
 '4 out of 5',
 '4.7 out of 5',
 '4.1 out of 5'

In [47]:
Noofrating

['1,032 ratings',
 '774 ratings',
 '4 ratings',
 '225 ratings',
 '774 ratings',
 '10 ratings',
 '940 ratings',
 '182 ratings',
 '50 ratings',
 '492 ratings',
 '160 ratings',
 '264 ratings',
 '708 ratings',
 '74 ratings',
 '85 ratings',
 '297 ratings',
 '155 ratings',
 '284 ratings',
 '321 ratings',
 '3 ratings',
 '378 ratings',
 '106 ratings',
 '139 ratings',
 '1 rating',
 '53 ratings',
 '111 ratings',
 '1,032 ratings',
 '774 ratings',
 '4 ratings',
 '225 ratings',
 '774 ratings',
 '10 ratings',
 '940 ratings',
 '182 ratings',
 '50 ratings',
 '492 ratings',
 '160 ratings',
 '264 ratings',
 '708 ratings',
 '74 ratings',
 '85 ratings',
 '297 ratings',
 '155 ratings',
 '284 ratings',
 '321 ratings',
 '3 ratings',
 '378 ratings',
 '106 ratings',
 '139 ratings',
 '1 rating',
 '53 ratings',
 '111 ratings',
 '1,032 ratings',
 '774 ratings',
 '4 ratings',
 '225 ratings',
 '774 ratings',
 '10 ratings',
 '940 ratings',
 '182 ratings',
 '50 ratings',
 '492 ratings',
 '160 ratings',
 '264 ratings'

In [48]:
Price

['₹62,990.00',
 '₹27,990.00',
 '₹32,500.00',
 '₹40,126.00',
 '₹27,990.00',
 '₹1,29,900.00',
 '₹1,30,990.00',
 '₹86,368.00',
 '₹69,999.00',
 '₹29,632.00',
 '₹31,990.00',
 '₹19,990.00',
 '₹41,890.00',
 '₹30,990.00',
 '₹99,492.00',
 '₹43,726.00',
 '₹31,550.00',
 '₹39,990.00',
 '₹23,490.00',
 '₹1,13,990.00',
 '-',
 '₹77,990.00',
 '₹36,990.00',
 '₹28,999.00',
 '₹64,090.00',
 '₹76,999.00',
 '₹62,990.00',
 '₹27,990.00',
 '₹32,500.00',
 '₹40,126.00',
 '₹27,990.00',
 '₹1,29,900.00',
 '₹1,30,990.00',
 '₹86,368.00',
 '₹69,999.00',
 '-',
 '₹31,990.00',
 '₹19,990.00',
 '₹41,890.00',
 '₹30,990.00',
 '₹99,492.00',
 '₹43,726.00',
 '₹31,550.00',
 '₹39,990.00',
 '₹23,490.00',
 '₹1,13,990.00',
 '-',
 '₹77,990.00',
 '₹36,990.00',
 '₹28,999.00',
 '₹64,090.00',
 '₹76,999.00',
 '₹62,990.00',
 '₹27,990.00',
 '₹32,500.00',
 '₹40,126.00',
 '₹27,990.00',
 '₹1,29,900.00',
 '₹1,30,990.00',
 '₹86,368.00',
 '₹69,999.00',
 '-',
 '₹31,990.00',
 '₹19,990.00',
 '₹41,890.00',
 '₹30,990.00',
 '₹99,492.00',
 '₹43,726.00',


In [49]:
Returnexchange

['7 Days Replacement',
 '-',
 '7 Days Replacement',
 '7 Days Replacement',
 '-',
 '7 Days Replacement',
 '-',
 '7 Days Replacement',
 '7 Days Replacement',
 '-',
 '7 Days Replacement',
 '-',
 '7 Days Replacement',
 '7 Days Replacement',
 '7 Days Replacement',
 '7 Days Replacement',
 '7 Days Replacement',
 '7 Days Replacement',
 '-',
 '7 Days Replacement',
 '7 Days Replacement',
 '7 Days Replacement',
 '7 Days Replacement',
 '7 Days Replacement',
 '7 Days Replacement',
 '7 Days Replacement',
 '7 Days Replacement',
 '-',
 '7 Days Replacement',
 '7 Days Replacement',
 '-',
 '7 Days Replacement',
 '-',
 '7 Days Replacement',
 '7 Days Replacement',
 '7 Days Replacement',
 '7 Days Replacement',
 '-',
 '7 Days Replacement',
 '7 Days Replacement',
 '7 Days Replacement',
 '7 Days Replacement',
 '7 Days Replacement',
 '7 Days Replacement',
 '-',
 '7 Days Replacement',
 '7 Days Replacement',
 '7 Days Replacement',
 '7 Days Replacement',
 '7 Days Replacement',
 '7 Days Replacement',
 '7 Days Repla

In [50]:
Expecteddelivery

['Tuesday, Sep 28',
 'Tuesday, Sep 28',
 'Friday, Oct 1',
 'Wednesday, Sep 29',
 'Tuesday, Sep 28',
 'Monday, Oct 4',
 'Tuesday, Sep 28',
 'Tuesday, Sep 28',
 'Wednesday, Sep 29',
 '-',
 'Tuesday, Sep 28',
 'Tuesday, Sep 28',
 'Tuesday, Sep 28',
 'Oct 21 - 26',
 'Friday, Oct 1',
 'Tuesday, Sep 28',
 'Friday, Oct 1',
 'Tuesday, Sep 28',
 'Friday, Oct 1',
 'Tuesday, Sep 28',
 '-',
 'Tuesday, Sep 28',
 'Tuesday, Sep 28',
 'Wednesday, Sep 29',
 'Wednesday, Oct 6',
 'Tuesday, Sep 28',
 'Tuesday, Sep 28',
 'Tuesday, Sep 28',
 'Friday, Oct 1',
 'Wednesday, Sep 29',
 'Tuesday, Sep 28',
 'Monday, Oct 4',
 'Tuesday, Sep 28',
 'Tuesday, Sep 28',
 'Wednesday, Sep 29',
 '-',
 'Tuesday, Sep 28',
 'Tuesday, Sep 28',
 'Tuesday, Sep 28',
 'Oct 21 - 26',
 'Friday, Oct 1',
 'Tuesday, Sep 28',
 'Friday, Oct 1',
 'Tuesday, Sep 28',
 'Friday, Oct 1',
 'Tuesday, Sep 28',
 '-',
 'Tuesday, Sep 28',
 'Tuesday, Sep 28',
 'Wednesday, Sep 29',
 'Wednesday, Oct 6',
 'Tuesday, Sep 28',
 'Tuesday, Sep 28',
 'Tuesday,

In [51]:
Availibility

['In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'Only 1 left in stock.',
 'In stock.',
 'Only 1 left in stock.',
 'In stock.',
 'Only 1 left in stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock on October 19, 2021.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 '',
 'In stock.',
 'In stock.',
 'In stock.',
 'Only 1 left in stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'Only 1 left in stock.',
 'In stock.',
 'Only 1 left in stock.',
 'In stock.',
 '',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock on October 19, 2021.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 '',
 'In stock.',
 'In stock.',
 'In stock.',
 'Only 1 left in stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'In stock.',
 'Only 1 left in stock.',
 'In stock.',
 'Only 1 left in stock.',
 'In stock.',
 '',
 'In stock.',
 'In s

In [52]:
Otherdetails

['Processor: 11th Gen Intel Core i5-1135G7 (up to 4.2 GHz with Intel Turbo Boost Technology, 8 MB L3 cache, 4 cores) Memory & Storage: 8 GB DDR4-3200 MHz RAM (2 x 4 GB), upgradable upto 16 GB| Storage: 512 GB PCIe NVMe M.2 SSD Display: 35.6 cm (14") diagonal, FHD (1920 x 1080), IPS, micro-edge, anti-glare, 250 nits, 45% NTSC Graphics & Networking: Intel Iris Xᵉ Graphics | Realtek RTL8822CE 802.11a/b/g/n/ac (2x2) Wi-Fi and Bluetooth 5 combo Operating System & Preinstalled Software: Pre-loaded Windows 10 Home with lifetime validity, FREE Upgrade to Windows 11* (when available, refer product description for more details) | Microsoft Office Home & Student 2019',
 'GOOGLE ASSISTANT: Voice-Enabled Google Assistant built-in, work faster and smarter without lifting a finger or switching screens. Ask questions, set reminders, play videos, control your home, and more. Make Google do it. BOOTS IN SECONDS: Powered by Chrome OS with automatic software updates so youâll always have the latest virus 

In [53]:
#printing length
print(len(Brandname),len(Nameofproduct),len(Rating),len(Noofrating),len(Price),len(Returnexchange),len(Expecteddelivery),len(Availibility),len(Otherdetails),len(Lurl))

78 78 78 78 78 78 78 78 78 78


In [54]:
#creating a dataframe

Laptops=pd.DataFrame({})
Laptops['Brand Name']=Brandname
Laptops['Name of Product']=Nameofproduct
Laptops['Rating']=Rating
Laptops['No of ratings']=Noofrating
Laptops['Price']=Price
Laptops['Return exchange']=Returnexchange
Laptops['Expected delivery']=Expecteddelivery
Laptops['Availibility']=Availibility
Laptops['Other Details']=Otherdetails
Laptops['Product URL']=Lurl

In [55]:
Laptops

,Brand Name,Name of Product,Rating,No of ratings,Price,Return exchange,Expected delivery,Availibility,Other Details,Product URL
0,Visit the HP Store,HP Pavilion 14 11th Gen Intel Core i5 14-inch ...,4.4 out of 5,"1,032 ratings","₹62,990.00",7 Days Replacement,"Tuesday, Sep 28",In stock.,Processor: 11th Gen Intel Core i5-1135G7 (up t...,https://www.amazon.in/gp/slredirect/picassoRed...
1,Visit the HP Store,HP Chromebook 14-inch (35.56 cms) Thin & Light...,3.8 out of 5,774 ratings,"₹27,990.00",-,"Tuesday, Sep 28",In stock.,GOOGLE ASSISTANT: Voice-Enabled Google Assista...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Visit the Acer Store,Acer Aspire 3 AMD Athlon Silver 3050U Dual-cor...,3.5 out of 5,4 ratings,"₹32,500.00",7 Days Replacement,"Friday, Oct 1",In stock.,Laptops Processor : AMD Athlon Silver 3050U du...,https://www.amazon.in/Acer-Aspire-Athlon-Dual-...
3,Visit the Dell Store,"Dell Inspiron 3502 15.6"" (39.62 cms) HD Displa...",4.2 out of 5,225 ratings,"₹40,126.00",7 Days Replacement,"Wednesday, Sep 29",In stock.,Free upgrade to Windows 11 when available. Dis...,https://www.amazon.in/Dell-Inspiron-Integrated...
4,Visit the HP Store,HP Chromebook 14-inch (35.56 cms) Thin & Light...,3.8 out of 5,774 ratings,"₹27,990.00",-,"Tuesday, Sep 28",In stock.,GOOGLE ASSISTANT: Voice-Enabled Google Assista...,https://www.amazon.in/HP-Chromebook-14a-na0003...
...,...,...,...,...,...,...,...,...,...,...
73,Visit the Lenovo Store,Lenovo IdeaPad S540 11th Gen Intel Core i7 13....,4.3 out of 5,106 ratings,"₹77,990.00",7 Days Replacement,"Tuesday, Sep 28",In stock.,Processor: 11th Gen Intel Core i7-1165G7 | Spe...,https://www.amazon.in/Lenovo-IdeaPad-Windows-G...
74,Visit the Acer Store,Acer Aspire 5 Intel Core i3 11th Generation 14...,4.3 out of 5,139 ratings,"₹36,990.00",7 Days Replacement,"Tuesday, Sep 28",In stock.,Free upgrade to Windows 11 when available* Pro...,https://www.amazon.in/Acer-Aspire-Generation-1...
75,Visit the Amazon Renewed Store,(Renewed) HP ProBook 640 G1 14-inch Laptop (Co...,5 out of 5,1 rating,"₹28,999.00",7 Days Replacement,"Wednesday, Sep 29",In stock.,-,https://www.amazon.in/HP-640-Upgradable-Graphi...
76,Visit the Lenovo Store,Lenovo IdeaPad Slim 3 Intel Core i5 10th Gen 1...,3.8 out of 5,53 ratings,-,7 Days Replacement,-,,Processor: Intel Core i5 10th Generation (i5-1...,https://www.amazon.in/gp/slredirect/picassoRed...


In [56]:
#converting into csv format
Laptops.to_csv("Laptops.csv")

In [57]:
#closing the driver
driver.close()

### 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [ ]:
#Scraping images of fruits and machine learning


#setting driver path
DRIVER_PATH = r"C:\chrome\chromedriver.exe"



def fetching_imageurls(query:str, max_links_to_fetch:100, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    imgurls = set()
    imgcount = 0
    resultsstart = 0
    while imgcount < max_links_to_fetch:
        scroll_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {resultsstart}:{number_results}")
        
        for img in thumbnail_results[resultsstart:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    imgurls.add(actual_image.get_attribute('src'))

            imgcount = len(imgurls)

            if len(imgurls) >= max_links_to_fetch:
                print(f"Found: {len(imgurls)} image links, done!")
                break
        else:
            print("Found:", len(imgurls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        resultsstart = len(thumbnail_results)

    return imgurls

def persistimage(folder_path:str,file_name:str,url:str):
    try:
        imgcontent = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        imgfile = io.BytesIO(imgcontent)
        image = Image.open(imgfile).convert('RGB')
        folder_path = os.path.join(folder_path,file_name)
        if os.path.exists(folder_path):
            file_path = os.path.join(folder_path,hashlib.sha1(imgcontent).hexdigest()[:10] + '.jpg')
        else:
            os.mkdir(folder_path)
            file_path = os.path.join(folder_path,hashlib.sha1(imgcontent).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

if __name__ == '__main__':
    wd = webdriver.Chrome(executable_path=DRIVER_PATH)
    queries = ["fruits","machine learning","cars"]  #set of querries
    for query in queries:
        wd.get('https://google.com')
        search_box = wd.find_element_by_css_selector('input.gLFyf')
        search_box.send_keys(query)
        links = fetching_imageurls(query,100,wd)
        #give images path
        images_path = r'C:\chrome\New folder'
        for i in links:
            persistimage(images_path,query,i)
    wd.quit()

Found: 100 search results. Extracting links from 0:100
Found: 100 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS9vJMxouHi_nw648VH1-ujWXdSSomvCB9TJg&usqp=CAU - as C:\chrome\New folder\fruits\f4f7f5f3e3.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSTGAL1dcvqVlRO9v4wQP1bnhUfULS33LnNrQ&usqp=CAU - as C:\chrome\New folder\fruits\9c3dea2579.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTDK35D457WbhLPKY1ahl43QAy4JGf4s4BKzQ&usqp=CAU - as C:\chrome\New folder\fruits\ead94c641d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSKfQuLBzNbAw4yd99gVQ1_Rc3kLUC8vM7ZNg&usqp=CAU - as C:\chrome\New folder\fruits\ba1e998c12.jpg
SUCCESS - saved https://images.theconversation.com/files/417973/original/file-20210826-2243-15s35b8.jpg?ixlib=rb-1.1.0&q=45&auto=format&w=1200&h=1200.0&fit=crop - as C:\chrome\New folder\fruits\65af223bfa.jpg
SUCCESS - saved https://www.muscleandfitness.com/w

### 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page.Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [4]:
#connecting to the webdriver
driver=webdriver.Chrome("chromedriver.exe")

In [5]:
#specifying the url of the webpage to be scraped
url="https://www.flipkart.com/"

In [6]:
#opening the webpage through our driver
driver.get(url)

In [7]:
# Taking input from user 
search = input("Input the string you want to search for:")

#generate a url from search string
url= driver.get("https://www.flipkart.com/search?q="+search+"&as=on&as-show=on&otracker=AS_Query_OrganicAutoSuggest_4_8_na_na_ps&otracker1=AS_Query_OrganicAutoSuggest_4_8_na_na_ps&as-pos=4&as-type=RECENT&suggestionId=samsung+m31&requestId=5c8b7098-e268-4146-b626-2617dd872a64&as-searchtext=samsung%20")

Input the string you want to search for:vivo V smartphone


In [8]:
#getting the list of url of vivo 
vivo_URL=[]


vivo=driver.find_elements_by_xpath("//div[@class='_2kHMtA']")
for i in vivo:
    vivo1=i.find_element_by_tag_name("a")
    vivo_URL.append(vivo1.get_attribute('href'))

In [9]:
vivo_URL

['https://www.flipkart.com/vivo-y51a-crystal-symphony-128-gb/p/itm72ed89f10c598?pid=MOBFZ2U4GURCS37U&lid=LSTMOBFZ2U4GURCS37UYYS09Z&marketplace=FLIPKART&q=vivo+V+smartphone&store=tyy%2F4io&srno=s_1_1&otracker=AS_Query_OrganicAutoSuggest_4_8_na_na_ps&otracker1=AS_Query_OrganicAutoSuggest_4_8_na_na_ps&fm=organic&iid=026b5d61-7b07-4913-affd-8846a580f8e3.MOBFZ2U4GURCS37U.SEARCH&ppt=None&ppn=None&ssid=3dz8xkcxgw0000001632745771424&qH=918e9a3d731a6088',
 'https://www.flipkart.com/vivo-y1s-olive-black-32-gb/p/itm44f0aa884b6d7?pid=MOBG3W3X7TAEXT3J&lid=LSTMOBG3W3X7TAEXT3JS8LLI3&marketplace=FLIPKART&q=vivo+V+smartphone&store=tyy%2F4io&srno=s_1_2&otracker=AS_Query_OrganicAutoSuggest_4_8_na_na_ps&otracker1=AS_Query_OrganicAutoSuggest_4_8_na_na_ps&fm=organic&iid=026b5d61-7b07-4913-affd-8846a580f8e3.MOBG3W3X7TAEXT3J.SEARCH&ppt=None&ppn=None&ssid=3dz8xkcxgw0000001632745771424&qH=918e9a3d731a6088',
 'https://www.flipkart.com/vivo-y51a-titanium-sapphire-128-gb/p/itm72ed89f10c598?pid=MOBG4FXCMHUNZBEY&lid

In [10]:
#checking the length
len(vivo_URL)

24

In [11]:
#going to the url of first phone through our driver
driver.get(vivo_URL[0])

In [12]:
#creating empty lists

brandname=[]
smartphonename=[]
colour=[]
ram=[]
rom=[]
primarycamera=[]
secondarycamera=[]
displaysize=[]
displayresolution=[]
processor=[]
processorcores=[]
batterycapacity=[]
price=[]

In [14]:
#running a loop to extract all the required information from the vivo_url
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
#running a loop to extract all the required information from the Phone_URL
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
for i in tqdm(vivo_URL):
    driver.get(i)
    driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
    
    #try:
    #driver.find_element_by_link_text('Read More').click()
    #except NoSuchElementException:
        #driver.find_element_by_css_selector('#container > div > div._2c7YLP.UtUXW0._6t1WkM._3HqJxg > div._1YokD2._2GoDe3 > div._1YokD2._3Mn1Gg.col-8-12 > div._1YokD2._3Mn1Gg > div:nth-child(4) > div > div:nth-child(2) > button').click()
    #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//button[text()="Read More"]'))).click()
    
    try:
        phone=driver.find_element_by_xpath('//*[@id="container"]/div/div[3]/div[1]/div[2]/div[2]/div/div[1]/h1/span').text
        smartphonename.append(phone)
    except NoSuchElementException:
        smartphonename.append("-")
    try:
      Colour=driver.find_element_by_xpath("//div[@class='_3k-BhJ']/table/tbody/tr[4]/td[2]/ul/li").text
      colour.append(Colour)  
    except NoSuchElementException:
        colour.append("-")
    try:
      RAM=driver.find_element_by_xpath("//div[@class='_3dtsli']/div[2]/div/div[4]/table/tbody/tr[2]/td[2]").text
      ram.append(RAM)
    except NoSuchElementException:
        ram.append("-")
        
    try:
      #
      #driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
      ROM=driver.find_element_by_xpath("//div[@class='_3dtsli']/div[2]/div/div[4]/table/tbody/tr[1]/td[2]").text
      rom.append(ROM)
    except NoSuchElementException:
        rom.append("-")
        
    try:
      #
      #driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
      prim=driver.find_element_by_xpath("//div[@class='_3dtsli']/div[2]/div/div[5]/table/tbody/tr[2]/td[2]").text
      primarycamera.append(prim)
    except NoSuchElementException:
        primarycamera.append("-")
        
    try:
        #
        #driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
        sec=driver.find_element_by_xpath("//div[@class='_3dtsli']/div[2]/div/div[5]/table/tbody/tr[5]/td[2]").text
        secondarycamera.append(sec)
    except NoSuchElementException:
        secondarycamera.append("-")
        
    try:
        #
        #driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
        ds=driver.find_element_by_xpath("//div[@class='_3dtsli']/div[2]/div/div[2]/table/tbody/tr[1]/td[2]").text
        displaysize.append(ds)
    except NoSuchElementException:
        displaysize.append("-")
        
    try:
        #
        #driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
        drs=driver.find_element_by_xpath("//div[@class='_3dtsli']/div[2]/div/div[2]/table/tbody/tr[2]/td[2]").text
        displayresolution.append(drs)
    except NoSuchElementException:
        displayresolution.append("-")
        
    try:
          #
          #driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
          pro_cessor=driver.find_element_by_xpath("//div[@class='_3dtsli']/div[2]/div/div[3]/table/tbody/tr[2]/td[2]").text
          processor.append(pro_cessor)
    except NoSuchElementException:
        processor.append("-")
        
    try:
        #
        #driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
        pro_cessor_cores=driver.find_element_by_xpath("//div[@class='_3dtsli']/div[2]/div/div[3]/table/tbody/tr[3]/td[2]").text
        processorcores.append(pro_cessor_cores)
    except NoSuchElementException:
        processorcores.append("-")
        
    try:
        #driver.get(Phone_URL[i])
        #driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
        batt=driver.find_element_by_xpath("//div[@class='_3dtsli']/div[2]/div/div[8]/table/tbody/tr[1]/td[2]").text
        batterycapacity.append(batt)
    except NoSuchElementException:
        batterycapacity.append("-")
        
    try:
        Price_=driver.find_element_by_css_selector('#container > div > div._2c7YLP.UtUXW0._6t1WkM._3HqJxg > div._1YokD2._2GoDe3 > div._1YokD2._3Mn1Gg.col-8-12 > div:nth-child(2) > div > div.dyC4hf > div.CEmiEU > div > div').text
        price.append(Price_)
    except NoSuchElementException:
        price.append("-")
        
    time.sleep(5)

100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [03:11<00:00,  7.99s/it]


In [15]:
smartphonename

['vivo Y51A (Crystal Symphony, 128 GB)  (8 GB RAM)',
 'vivo Y51A (Crystal Symphony, 128 GB)  (8 GB RAM)',
 'vivo Y1s (Olive Black, 32 GB)  (3 GB RAM)',
 'vivo Y51A (Titanium Sapphire, 128 GB)  (6 GB RAM)',
 'vivo Y1s (Aurora Blue, 32 GB)  (3 GB RAM)',
 'vivo Y51A (Crystal Symphony, 128 GB)  (6 GB RAM)',
 'vivo Y91i (Fusion Black, 32 GB)  (2 GB RAM)',
 'vivo V20 Pro (Sunset Melody, 128 GB)  (8 GB RAM)',
 'vivo V20 Pro (Midnight Jazz, 128 GB)  (8 GB RAM)',
 'vivo V17 (Glacier ice white, 128 GB)  (8 GB RAM)',
 'vivo Z1x (Fusion Blue, 64 GB)  (6 GB RAM)',
 'vivo X50 Pro (Alpha Grey, 256 GB)  (8 GB RAM)',
 'vivo V9 (Pearl Black, 64 GB)  (4 GB RAM)',
 'vivo V19 (Mystic Silver, 128 GB)  (8 GB RAM)',
 'vivo V19 (Mystic Silver, 256 GB)  (8 GB RAM)',
 'vivo V15 Pro (Topaz Blue, 128 GB)  (6 GB RAM)',
 'vivo Y91i (Ocean Blue, 16 GB)  (2 GB RAM)',
 'vivo Y91i (Fusion Black, 16 GB)  (2 GB RAM)',
 'vivo Y81 (Gold, 32 GB)  (3 GB RAM)',
 'vivo V17Pro (Midnight Ocean Black, 128 GB)  (8 GB RAM)',
 'vivo 

In [16]:
colour

['Crystal Symphony',
 'Crystal Symphony',
 'Olive Black',
 'Titanium Sapphire',
 'Aurora Blue',
 'Crystal Symphony',
 'Fusion Black',
 'Sunset Melody',
 'Midnight Jazz',
 'Glacier ice white',
 'Fusion Blue',
 'Alpha Grey',
 'Pearl Black',
 'Mystic Silver',
 'Mystic Silver',
 'Topaz Blue',
 'Ocean Blue',
 'Fusion Black',
 'Gold',
 'Midnight Ocean Black',
 'Topaz Blue',
 'Ocean Blue',
 'Black',
 'Starry Night Black',
 'Dazzling Gold']

In [17]:
ram

['8 GB',
 '8 GB',
 '3 GB',
 '6 GB',
 '3 GB',
 '6 GB',
 '2 GB',
 '8 GB',
 '8 GB',
 '8 GB',
 '6 GB',
 '8 GB',
 '4 GB',
 '8 GB',
 '8 GB',
 '6 GB',
 '2 GB',
 '2 GB',
 '3 GB',
 '8 GB',
 '8 GB',
 '2 GB',
 '3 GB',
 '6 GB',
 '6 GB']

In [18]:
rom

['128 GB',
 '128 GB',
 '32 GB',
 '128 GB',
 '32 GB',
 '128 GB',
 '32 GB',
 '128 GB',
 '128 GB',
 '128 GB',
 '64 GB',
 '256 GB',
 '64 GB',
 '128 GB',
 '256 GB',
 '128 GB',
 '16 GB',
 '16 GB',
 '32 GB',
 '128 GB',
 '128 GB',
 '32 GB',
 '32 GB',
 '64 GB',
 '64 GB']

In [19]:
primarycamera

['48MP + 8MP + 2MP',
 '13MP Rear Camera',
 '48MP + 8MP + 2MP',
 '13MP Rear Camera',
 '48MP + 8MP + 2MP',
 '13MP Rear Camera',
 '64MP + 8MP + 2MP',
 '64MP + 8MP + 2MP',
 '48MP + 8MP + 2MP + 2MP',
 '48MP + 8MP + 2MP',
 '48MP + 13MP + 8MP + 8MP',
 '16MP + 5MP',
 '48MP + 8MP + 2MP + 2MP',
 '48MP + 8MP + 2MP + 2MP',
 '48MP + 5MP + 8MP',
 '13MP Rear Camera',
 '13MP Rear Camera',
 '13MP Rear Camera',
 '48MP + 13MP + 8MP + 2MP',
 '48 Million Quad Pixel Sensor (12 Million Effective Pixel) + 8MP + 5MP, AI Triple Rear Camera',
 '13MP Rear Camera',
 '13MP Rear Camera',
 '12MP + 5MP',
 '12MP + 5MP']

In [20]:
secondarycamera

['16MP Front Camera',
 '5MP Front Camera',
 '16MP Front Camera',
 '5MP Front Camera',
 '16MP Front Camera',
 '5MP Front Camera',
 '44MP + 8MP Dual Front Camera',
 '44MP + 8MP Dual Front Camera',
 '32MP Front Camera',
 '32MP (f2.0), AI Filters, AR stikcers, Portrait light effects, Live Photo, AI Beauty, Fun Video etc.',
 '32MP Front Camera',
 '24MP Front Camera',
 '32MP + 8MP Dual Front Camera',
 '32MP + 8MP Dual Front Camera',
 '32MP Front Camera',
 '5MP Front Camera',
 '5MP Front Camera',
 '5MP Front Camera',
 '32MP + 8MP Dual Front Camera',
 '32MP Front Camera',
 '5MP Front Camera',
 '5MP Front Camera',
 '25MP Front Camera',
 '25MP Front Camera']

In [21]:
displaysize

['16.71 cm (6.58 inch)',
 '15.8 cm (6.22 inch)',
 '16.71 cm (6.58 inch)',
 '15.8 cm (6.22 inch)',
 '16.71 cm (6.58 inch)',
 '15.8 cm (6.22 inch)',
 '16.36 cm (6.44 inch)',
 '16.36 cm (6.44 inch)',
 '16.36 cm (6.44 inch)',
 '16.21 cm (6.38 inch)',
 '16.66 cm (6.56 inch)',
 '16.0 cm (6.3 inch)',
 '16.36 cm (6.44 inch)',
 '16.36 cm (6.44 inch)',
 '16.23 cm (6.39 inch)',
 '15.8 cm (6.22 inch)',
 '15.8 cm (6.22 inch)',
 '15.49 cm (6.1 inch)',
 '16.36 cm (6.44 inch)',
 '16.23 cm (6.39 inch)',
 '15.8 cm (6.22 inch)',
 '15.49 cm (6.1 inch)',
 '16.28 cm (6.41 inch)',
 '16.28 cm (6.41 inch)']

In [22]:
displayresolution

['2408 × 1080 Pixels',
 '1520 x 720 Pixels',
 '2408 x 1080 Pixels',
 '1520 x 720 Pixels',
 '2408 x 1080 Pixels',
 '1520 x 720 pixels',
 '2400 x 1080 Pixels',
 '2400 x 1080 Pixels',
 '2400 x 1080 Pixels',
 '2340 x 1080 Pixels',
 '2376 x 1080 Pixels',
 '2280 x 1080 Pixels',
 '2400 × 1080 Pixels',
 '2400 × 1080 Pixels',
 '2340 x 1080 pixels',
 '1520 x 720 pixels',
 '1520 x 720 pixels',
 '1520 x 720 Pixels',
 '2400 x 1080 Pixels',
 '2340 x 1080 pixels',
 '1520 x 720 pixels',
 '1520 x 720 Pixels',
 '2340 x 1080 pixels',
 '2340 x 1080 pixels']

In [23]:
processor

['Qualcomm Snapdragon 662',
 'MediaTek Helio P35',
 'Qualcomm Snapdragon 662',
 'MediaTek Helio P35',
 'Qualcomm Snapdragon 662',
 'MTK Helio P22',
 'Qualcomm Snapdragon 765G',
 'Qualcomm Snapdragon 765G',
 'Qualcomm Snapdragon 675AIE',
 'Qualcomm Snapdragon 712 AIE',
 'Qualcomm Snapdragon 765G',
 'Qualcomm Snapdragon 626',
 'Qualcomm Snapdragon 712 AIE',
 'Qualcomm Snapdragon 712 AIE',
 'Qualcomm Snapdragon 675AIE Octa Core 2.0GHz',
 'MTK Helio P22',
 'MTK Helio P22',
 'MT6762 Octa Core (Helio P22)',
 'Qualcomm Snapdragon 675 AIE',
 'Qualcomm Snapdragon 675AIE Octa Core 2.0GHz',
 'MTK Helio P22',
 'MT6762 Octa Core (Helio P22)',
 '2.0 GHz Qualcomm Snapdragon 660 AIE',
 '2.0 GHz Qualcomm Snapdragon 660 AIE']

In [24]:
processorcores

['Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core',
 'Octa Core']

In [25]:
batterycapacity

['Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'Yes',
 'WAV, MP3, MP2, AMR-NB, AMR-WB, MIDI, Vorbis, APE, FLAC',
 'WAV, MP3, MP2, AMR-NB, AMR-WB, MIDI, Vorbis, APE, FLAC',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes']

In [26]:
price

['₹17,990',
 '₹9,490',
 '₹16,990',
 '₹9,490',
 '₹16,990',
 '₹7,990',
 '₹29,990',
 '₹27,990',
 '₹27,990',
 '₹16,990',
 '₹49,990',
 '₹26,000',
 '₹24,990',
 '₹29,990',
 '₹21,099',
 '₹9,811',
 '₹9,934',
 '₹12,900',
 '₹32,990',
 '₹28,499',
 '₹10,097',
 '₹15,000',
 '₹22,654',
 '₹25,990']

In [27]:
#extracting brand_name from smartphone_name
brandname = [i.split(maxsplit=1)[0] for i in smartphonename]

In [28]:
brandname

['vivo',
 'vivo',
 'vivo',
 'vivo',
 'vivo',
 'vivo',
 'vivo',
 'vivo',
 'vivo',
 'vivo',
 'vivo',
 'vivo',
 'vivo',
 'vivo',
 'vivo',
 'vivo',
 'vivo',
 'vivo',
 'vivo',
 'vivo',
 'vivo',
 'vivo',
 'vivo',
 'vivo',
 'vivo']

In [29]:
#printing lengths
print(len(brandname),len(smartphonename),len(colour),len(ram),len(rom),len(primarycamera),len(secondarycamera),len(displaysize),len(displayresolution),len(processor),len(processorcores),len(batterycapacity),len(price),len(vivo_URL))

25 25 25 25 25 24 24 24 24 24 24 24 24 24


In [32]:
#creating a dataframe

Smphone=pd.DataFrame({})
Smphone['Brand Name']=brandname[0:24]
Smphone['Smartphone Name']=smartphonename[0:24]
Smphone['Colour']=colour[0:24]
Smphone['Ram']=ram[0:24]
Smphone['Rom']=rom[0:24]
Smphone['Primary Camera']=primarycamera
Smphone['Secondary Camera']=secondarycamera
Smphone['Display Size']=displaysize
Smphone['Display Resolution']=displayresolution
Smphone['Processor']=processor
Smphone['Processor Cores']=processorcores
Smphone['Battery Capacity']=batterycapacity
Smphone['Price']=price
Smphone['Phone URL']=vivo_URL

In [33]:
Smphone

,Brand Name,Smartphone Name,Colour,Ram,Rom,Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Price,Phone URL
0,vivo,"vivo Y51A (Crystal Symphony, 128 GB) (8 GB RAM)",Crystal Symphony,8 GB,128 GB,48MP + 8MP + 2MP,16MP Front Camera,16.71 cm (6.58 inch),2408 × 1080 Pixels,Qualcomm Snapdragon 662,Octa Core,Yes,"₹17,990",https://www.flipkart.com/vivo-y51a-crystal-sym...
1,vivo,"vivo Y51A (Crystal Symphony, 128 GB) (8 GB RAM)",Crystal Symphony,8 GB,128 GB,13MP Rear Camera,5MP Front Camera,15.8 cm (6.22 inch),1520 x 720 Pixels,MediaTek Helio P35,Octa Core,Yes,"₹9,490",https://www.flipkart.com/vivo-y1s-olive-black-...
2,vivo,"vivo Y1s (Olive Black, 32 GB) (3 GB RAM)",Olive Black,3 GB,32 GB,48MP + 8MP + 2MP,16MP Front Camera,16.71 cm (6.58 inch),2408 x 1080 Pixels,Qualcomm Snapdragon 662,Octa Core,Yes,"₹16,990",https://www.flipkart.com/vivo-y51a-titanium-sa...
3,vivo,"vivo Y51A (Titanium Sapphire, 128 GB) (6 GB RAM)",Titanium Sapphire,6 GB,128 GB,13MP Rear Camera,5MP Front Camera,15.8 cm (6.22 inch),1520 x 720 Pixels,MediaTek Helio P35,Octa Core,Yes,"₹9,490",https://www.flipkart.com/vivo-y1s-aurora-blue-...
4,vivo,"vivo Y1s (Aurora Blue, 32 GB) (3 GB RAM)",Aurora Blue,3 GB,32 GB,48MP + 8MP + 2MP,16MP Front Camera,16.71 cm (6.58 inch),2408 x 1080 Pixels,Qualcomm Snapdragon 662,Octa Core,Yes,"₹16,990",https://www.flipkart.com/vivo-y51a-crystal-sym...
5,vivo,"vivo Y51A (Crystal Symphony, 128 GB) (6 GB RAM)",Crystal Symphony,6 GB,128 GB,13MP Rear Camera,5MP Front Camera,15.8 cm (6.22 inch),1520 x 720 pixels,MTK Helio P22,Octa Core,Yes,"₹7,990",https://www.flipkart.com/vivo-y91i-fusion-blac...
6,vivo,"vivo Y91i (Fusion Black, 32 GB) (2 GB RAM)",Fusion Black,2 GB,32 GB,64MP + 8MP + 2MP,44MP + 8MP Dual Front Camera,16.36 cm (6.44 inch),2400 x 1080 Pixels,Qualcomm Snapdragon 765G,Octa Core,Yes,"₹29,990",https://www.flipkart.com/vivo-v20-pro-sunset-m...
7,vivo,"vivo V20 Pro (Sunset Melody, 128 GB) (8 GB RAM)",Sunset Melody,8 GB,128 GB,64MP + 8MP + 2MP,44MP + 8MP Dual Front Camera,16.36 cm (6.44 inch),2400 x 1080 Pixels,Qualcomm Snapdragon 765G,Octa Core,Yes,"₹27,990",https://www.flipkart.com/vivo-v20-pro-midnight...
8,vivo,"vivo V20 Pro (Midnight Jazz, 128 GB) (8 GB RAM)",Midnight Jazz,8 GB,128 GB,48MP + 8MP + 2MP + 2MP,32MP Front Camera,16.36 cm (6.44 inch),2400 x 1080 Pixels,Qualcomm Snapdragon 675AIE,Octa Core,Yes,"₹27,990",https://www.flipkart.com/vivo-v17-glacier-ice-...
9,vivo,"vivo V17 (Glacier ice white, 128 GB) (8 GB RAM)",Glacier ice white,8 GB,128 GB,48MP + 8MP + 2MP,"32MP (f2.0), AI Filters, AR stikcers, Portrait...",16.21 cm (6.38 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 712 AIE,Octa Core,No,"₹16,990",https://www.flipkart.com/vivo-z1x-fusion-blue-...


In [34]:
#converting into csv format
Smphone.to_csv("Smartphone.csv")

In [35]:
#closing the driver
driver.close()

### 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [59]:
driver=webdriver.Chrome("chromedriver.exe")

In [60]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import re

loc = "chennai"
url = 'https://www.google.com/maps/search/{}'.format(loc)

driver.get(url)
r = re.compile(r'center=(-?\d+\.\d+)%2C(-?\d+\.\d+)')
Latitude_long = r.findall(driver.page_source)


print(Latitude_long)

driver.quit()

[('13.04752545', '80.20901175'), ('13.04752545', '80.20901175')]


In [61]:
coord=Latitude_long[0:1]
coord

[('13.04752545', '80.20901175')]

### 6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in

In [36]:
#connecting to the webdriver
driver=webdriver.Chrome("chromedriver.exe")

In [38]:
#specifying the url of the webpage to be scraped
url = "https://trak.in/"

In [39]:
#opening the url through our driver
driver.get(url)

In [40]:
button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(button)

In [41]:
#creating empty lists

Date=[]
Startup=[]
Vertical=[]
Subvertical=[]
Location=[]
Investors=[]
Investmenttype=[]
Amount=[]

In [43]:
for i in range(48,51):
    driver.find_element_by_xpath('//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()

    dt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        Date.append(d.text)

    sn = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        Startup.append(n.text)
    
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        Vertical.append(n.text)
    
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        Subvertical.append(s.text)

    loc = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        Location.append(l.text)
    
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        Investors.append(n.text)
    
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        Investmenttype.append(n.text)
    
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        Amount.append(a.text)

In [53]:
Date

['15/07/2020',
 '16/07/2020',
 '16/07/2020',
 '14/07/2020',
 '13/07/2020',
 '09/07/2020',
 '10/07/2020',
 '15/07/2020',
 '14/07/2020',
 '13/07/2020',
 '15/07/2020',
 '16/07/2020',
 '16/07/2020',
 '14/07/2020',
 '13/07/2020',
 '09/07/2020',
 '10/07/2020',
 '15/07/2020',
 '14/07/2020',
 '13/07/2020',
 '15/08/2020',
 '13/08/2020',
 '13/08/2020',
 '04/08/2020',
 '11/08/2020',
 '12/08/2020',
 '14/08/2020',
 '10/08/2020',
 '07/08/2020',
 '13/08/2020',
 '03/08/2020',
 '04/08/2020',
 '08/09/2020',
 '12/09/2020',
 '09/09/2020',
 '02/09/2020',
 '09/09/2020',
 '07/09/2020',
 '07/09/2020',
 '31/08/2020',
 '31/08/2020',
 '03/09/2020',
 '08/09/2020']

In [54]:
Startup

['Flipkart',
 'Vedantu',
 'Crio',
 'goDutch',
 'Mystifly',
 'JetSynthesys',
 'gigIndia',
 'PumPumPum',
 'FLYX',
 'Open Appliances Pvt. Ltd.',
 'Flipkart',
 'Vedantu',
 'Crio',
 'goDutch',
 'Mystifly',
 'JetSynthesys',
 'gigIndia',
 'PumPumPum',
 'FLYX',
 'Open Appliances Pvt. Ltd.',
 'Practo',
 'Medlife',
 'HungerBox',
 'Dunzo',
 'Terra.do',
 'Classplus',
 'Niyo',
 'ZestMoney',
 'FreshToHome',
 'Eduvanz',
 'CrowdPouch',
 'DrinkPrime',
 'Byju’s',
 'mCaffeine',
 'Qshala',
 'Winzo',
 'Hippo Video',
 'Melorra',
 '1mg',
 'mfine',
 'Apna',
 'Railofy',
 'Cell Propulsion']

In [55]:
Vertical

['E-commerce',
 'EduTech',
 'EduTech',
 'FinTech',
 'Airfare Marketplace',
 'Gaming and Entertainment',
 'Marketplace',
 'Automotive Rental',
 'OTT Player',
 'Information Technology',
 'E-commerce',
 'EduTech',
 'EduTech',
 'FinTech',
 'Airfare Marketplace',
 'Gaming and Entertainment',
 'Marketplace',
 'Automotive Rental',
 'OTT Player',
 'Information Technology',
 'HealthTech',
 'E-commerce',
 'FoodTech',
 'Hyper-local Logistics',
 'EduTech',
 'EduTech',
 'FinTech',
 'FinTech',
 'E-commerce',
 'FinTech',
 'FinTech',
 'Water Purification',
 'EduTech',
 'Personal Care',
 'EduTech',
 'Online Gaming',
 'Video Customer Experience(CX) Platform',
 'E-commerce',
 'E-commerce',
 'HealthTech',
 'Human Resources',
 'Transportation',
 'Automobile']

In [56]:
Subvertical

['E-commerce',
 'Online Tutoring',
 'Learning Platform for Developers',
 'Group Payments',
 'Ticketing, Airline Retailing, and Post-Ticketing Services',
 'Gaming and Entertainment',
 'Crowd Sourcing, Freelance',
 'Used Car-leasing platform',
 'Streaming Social Network',
 'Internet-of-Things Security Solutions',
 'Health care and Wellness',
 'Online Pharmacy',
 'Online Food Delivery Service',
 'Online Delivery Services',
 'Online Climate School, E-learning',
 'E-learning, Online Tutoring',
 'Financial Services',
 'Financial Services',
 'Food Delivery',
 'Financial Services',
 'Financial Services',
 'Water Purification',
 'Online Tutoring',
 'Skincare & Haircare',
 'Online Curiosity Platform for Kids',
 'Online Gaming',
 'Video Customer Experience(CX) Platform',
 'Online Jewelry Store',
 'Online Pharmacy',
 'On-Demand Healthcare Services',
 'Recruitment Platform',
 'WL & RAC protection platform',
 'Electric Mobility Solutions']

In [57]:
Location

['Bangalore',
 'Bangalore',
 'Bangalore',
 'Mumbai',
 'Singapore and Bangalore',
 'Pune',
 'Pune',
 'Gurgaon',
 'New York and Delhi',
 'Bangalore',
 'Bangalore',
 'Bangalore',
 'Bangalore',
 'Bangalore',
 'Stanford, California,',
 'Noida',
 'Bangalore',
 'Bangalore',
 'Bangalore',
 'Mumbai',
 'Bangalore',
 'Bangalore',
 'Bangalore',
 'Mumbai',
 'Bangalore',
 'New Delhi',
 'Newark, Delaware, United States of Amercia',
 'Bangalore',
 'Gurgaon',
 'Bangalore',
 'Bangalore',
 'Mumbai',
 'Bangalore']

In [58]:
Investors

['Walmart Inc',
 'Coatue Management',
 '021 Capital',
 'Matrix India,Y Combinator, Global Founders Capital, Soma Capital, and VentureSouq.',
 'Recruit Co. Ltd.',
 'Adar Poonawalla and Kris Gopalakrishnan.',
 'Incubate Fund India and Beyond Next Ventures',
 'Early Adapters Syndicate',
 'Raj Mishra, founder of AIT Global Inc',
 'Unicorn India Ventures',
 'A1A Company',
 'Prasid Uno Family Trust and SC Credit Fund',
 'One97, Sabre Partners Trust, Pratithi Investment Trust, and Srihari Kumar',
 'Existing Backers',
 'Stanford Angels and Entrepreneurs (India), BEENEXT Emerging Asia, Rainmatter Capital',
 'Falcon Edge',
 'Niyo Solutions Inc.',
 'Primrose Hills Ventures',
 'Ascent Capital',
 'Sequoia India, Unitus',
 'Elina Investments Pvt. Ltd',
 'Sequoia Surge, ON Mauritius',
 'Silver Lake, Tiger Global, General Atlantic and Owl Ventures',
 'Amicus Capital Private Equity I LLP, Amicus Capital Partners India Fund I and RP-SG Venture Fund 1',
 'Rainmatter Capital',
 'Kalaari Capital Partners, 

In [59]:
Investmenttype

['M&A',
 'Series D',
 'pre-Series A',
 'Seed',
 'pre-Series B',
 'Venture-Series Unknown',
 'pre-Series A',
 'Seed',
 'pre-Seed',
 'Venture-Series Unknown',
 'Series F',
 '',
 'Series D1',
 'In Progress',
 'Seed',
 'In Progress',
 '',
 '',
 'Venture',
 'Series A',
 'Angel',
 'Pre-Series A',
 'Private Equity',
 'Series B',
 'Angel',
 'Series B',
 'Series A',
 'Debt Financing',
 'In Progress',
 'Series B',
 'Series A',
 'Seed',
 'pre-Series A']

In [60]:
Amount

['1,200,000,000',
 '100,000,000',
 '934,160',
 '1,700,000',
 '3,300,000',
 '400,000',
 '974,200',
 '292,800',
 '200,000',
 '500,000',
 '32,000,000',
 '23,000,000',
 '1,560,000',
 '30,000,000',
 '1,400,000',
 'upto 15,000,000',
 '6,000,000',
 '10,670,000',
 '16,200,000',
 '5,000,000',
 'NA',
 '2,880,000',
 '500,000,000',
 '3,000,000',
 '370,000',
 '15,500,000',
 '4,500,000',
 'upto 8,900,000',
 '100,000,000',
 '5,400,000',
 '8,000,000',
 '950,000',
 'NA']

In [47]:
#printing lengths
print(len(Date),len(Startup),len(Vertical),len(Subvertical),len(Location),len(Investors),len(Investmenttype),len(Amount))

43 43 43 33 33 33 33 33


In [48]:
#creating a dataframe

funding=pd.DataFrame({})
funding['Date']=Date[0:33]
funding['Startup']=Startup[0:33] 
funding['Vertical']=Vertical[0:33]
funding['SubVertical']=Subvertical
funding['Location']=Location
funding['Investors']=Investors
funding['Investment Type']=Investmenttype
funding['Amount']=Amount

In [49]:
funding

,Date,Startup,Vertical,SubVertical,Location,Investors,Investment Type,Amount
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


In [52]:
#converting into csv format
funding.to_csv("Funding.csv")

In [61]:
#closing the driver
driver.close()

### 7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in. 

In [62]:
#connecting to the webdriver
driver=webdriver.Chrome("chromedriver.exe")

In [63]:
#specifying the url of the webpage to be scraped
url = "https://www.digit.in/"

In [64]:
#opening the url through our driver
driver.get(url)
time.sleep(3)

In [65]:
#lets search for best laptop
bestgaming_lap = driver.find_element_by_xpath("//div[@class='listing_container']//ul//li[9]").click()
time.sleep(4)

In [67]:
#lets fetch info about laptops
lapname = []
opstm = []
display = []
processor = []
memory = []
weight = []
dimensions = []
graphprocessor = []
prices = []

In [68]:
name = driver.find_elements_by_xpath("//table[@id='summtable']//tr//td[1]")
for i in name:
    lapname.append(i.text)
#fetching info related to operating system
try:
    os = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[3]//td[3]")
    for i in os:
        opstm.append(i.text)
except NoSuchElementException:
    pass
try:
    disp = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[4]//td[3]")
    for i in disp:
        display.append(i.text)
except NoSuchElementException:
    pass
try:
    pro = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[5]//td[3]")
    for i in pro:
        processor.append(i.text)
except NoSuchElementException:
    pass
try:
    mem = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[6]//td[3]")
    for i in mem:
        memory.append(i.text)
except NoSuchElementException:
    pass
try:
    wgt = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[7]//td[3]")
    for i in wgt:
        weight.append(i.text)
except NoSuchElementException:
    pass
try:
    dim = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[8]//td[3]")
    for i in dim:
        dimensions.append(i.text)
except NoSuchElementException:
    pass
try:
    gpro = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[9]//td[3]")
    for i in gpro:
        graphprocessor.append(i.text)
except NoSuchElementException:
    pass
try:
    price = driver.find_elements_by_xpath("//td[@class='smprice']")
    for i in price:
        prices.append(i.text.replace('₹','Rs '))
except NoSuchElementException:
    pass

In [69]:
#creating dataframe
data = list(zip(lapname,opstm,display,processor,memory,weight,dimensions,graphprocessor,prices))
df = pd.DataFrame(data, columns = ["Laptop Name","Operating System","Display","Processor","Memory","Weight(Kg)","Dimensions","Graphic-Processor","Price"])
df

,Laptop Name,Operating System,Display,Processor,Memory,Weight(Kg),Dimensions,Graphic-Processor,Price
0,Alienware Area 51M R2,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,N/A
1,Alienware m15 R3,Windows 10 Home,"15.6"" (3840 x 2160)",10th Generation Intel® Core™ i9-10980HK | NA,1 TB SSD/16 GBGB DDR4,NA,NA,NA,"Rs 341,990"
2,ASUS ROG Strix Scar 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,N/A
3,Asus ROG Zephyrus G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,"Rs 164,990"
4,Lenovo Legion 5i,Windows 10 Pro,"15.6"" (1920 x 1080)",10th Generation Intel® Core™ i5-10300H | 2.50 GHz,1 TB SSD/16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB,"Rs 76,988"
5,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,"Rs 185,000"
6,Acer Aspire 7 gaming,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,"Rs 64,370"


In [70]:
#converting into csv format
df.to_csv("Top10_Laptops.csv")

In [71]:
#closing the driver
driver.close()

# 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [4]:
driver=webdriver.Chrome("chromedriver.exe")

In [5]:
driver.get('https://www.forbes.com/?sh=1d7fec302254')

In [6]:
driver.find_element_by_xpath('//button[@class="icon--hamburger"]').click()
time.sleep(1)

In [7]:
from selenium.webdriver.common.action_chains import ActionChains
action = ActionChains(driver)
head= driver.find_element_by_xpath('//a[@class="header__title"]')
action.move_to_element(head).perform()
time.sleep(1)
sec= driver.find_element_by_xpath('//a[@class="section__link"]')
sec.click()

In [8]:
rank_ = []
name_ = []
networth_ = []
age_ = []
citizenship_ = []
source_ = [] 
industry_ = []

In [9]:
for _ in range(0,15):
    nme = driver.find_elements_by_xpath('//div[@class="personName"]')
    Rnk = driver.find_elements_by_xpath('//div[@class="rank"]')
    wort = driver.find_elements_by_xpath('//div[@class="netWorth"]')
    Ag = driver.find_elements_by_xpath('//div[@class="age"]')
    citi_ = driver.find_elements_by_xpath('//div[@class="countryOfCitizenship"]')
    Sou = driver.find_elements_by_xpath('//div[@class="source-column"]')
    Ind = driver.find_elements_by_xpath('//div[@class="category"]')
    
    for r in Rnk:
        rank_.append(r.text.replace('.',''))
        
    for n in nme:
        name_.append(n.text)
    
    for w in wort:
        networth_.append(w.text)
        
    for a in Ag:
        age_.append(a.text)
        
    for c in citi_:
        citizenship_.append(c.text)
        
    for s in Sou:
        source_.append(s.text)
    
    for i in Ind:
        industry_.append(i.text)
        
    
        
    try:
        nxt = driver.find_element_by_xpath('//button[@class="pagination-btn pagination-btn--next "]')
        nxt.click()
    except:
        pass

In [13]:
name_

['Jeff Bezos',
 'Elon Musk',
 'Bernard Arnault & family',
 'Bill Gates',
 'Mark Zuckerberg',
 'Warren Buffett',
 'Larry Ellison',
 'Larry Page',
 'Sergey Brin',
 'Mukesh Ambani',
 'Amancio Ortega',
 'Francoise Bettencourt Meyers & family',
 'Zhong Shanshan',
 'Steve Ballmer',
 'Ma Huateng',
 'Carlos Slim Helu & family',
 'Alice Walton',
 'Jim Walton',
 'Rob Walton',
 'Michael Bloomberg',
 'Colin Zheng Huang',
 'MacKenzie Scott',
 'Daniel Gilbert',
 'Gautam Adani & family',
 'Phil Knight & family',
 'Jack Ma',
 'Charles Koch',
 'Julia Koch & family',
 'Masayoshi Son',
 'Michael Dell',
 'Tadashi Yanai & family',
 'François Pinault & family',
 'David Thomson & family',
 'Beate Heister & Karl Albrecht Jr.',
 'Wang Wei',
 'Miriam Adelson',
 'He Xiangjian',
 'Dieter Schwarz',
 'Zhang Yiming',
 'Giovanni Ferrero',
 'Alain Wertheimer',
 'Gerard Wertheimer',
 'Li Ka-shing',
 'Qin Yinglin & family',
 'William Lei Ding',
 'Len Blavatnik',
 'Lee Shau Kee',
 'Jacqueline Mars',
 'John Mars',
 'Yang 

In [14]:
rank_

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '27',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '41',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '48',
 '50',
 '51',
 '52',
 '53',
 '53',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '62',
 '64',
 '65',
 '66',
 '66',
 '68',
 '69',
 '70',
 '71',
 '71',
 '73',
 '74',
 '75',
 '75',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '89',
 '89',
 '92',
 '93',
 '94',
 '95',
 '96',
 '96',
 '98',
 '99',
 '100',
 '100',
 '102',
 '103',
 '104',
 '105',
 '105',
 '107',
 '107',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '119',
 '121',
 '121',
 '121',
 '124',
 '124',
 '126',
 '127',
 '128',
 '128',
 '128',
 '131',
 '132',
 '133',
 '133',
 '133',
 '133',
 '137',
 '138',
 '13

In [15]:
networth_

['$177 B',
 '$151 B',
 '$150 B',
 '$124 B',
 '$97 B',
 '$96 B',
 '$93 B',
 '$91.5 B',
 '$89 B',
 '$84.5 B',
 '$77 B',
 '$73.6 B',
 '$68.9 B',
 '$68.7 B',
 '$65.8 B',
 '$62.8 B',
 '$61.8 B',
 '$60.2 B',
 '$59.5 B',
 '$59 B',
 '$55.3 B',
 '$53 B',
 '$51.9 B',
 '$50.5 B',
 '$49.9 B',
 '$48.4 B',
 '$46.4 B',
 '$46.4 B',
 '$45.4 B',
 '$45.1 B',
 '$44.1 B',
 '$42.3 B',
 '$41.8 B',
 '$39.2 B',
 '$39 B',
 '$38.2 B',
 '$37.7 B',
 '$36.9 B',
 '$35.6 B',
 '$35.1 B',
 '$34.5 B',
 '$34.5 B',
 '$33.7 B',
 '$33.5 B',
 '$33 B',
 '$32 B',
 '$31.7 B',
 '$31.3 B',
 '$31.3 B',
 '$29.6 B',
 '$29.1 B',
 '$28.4 B',
 '$27.7 B',
 '$27.7 B',
 '$27 B',
 '$26.9 B',
 '$26.4 B',
 '$26.3 B',
 '$26.2 B',
 '$26.1 B',
 '$25.9 B',
 '$25.8 B',
 '$25.8 B',
 '$25.5 B',
 '$25 B',
 '$24.9 B',
 '$24.9 B',
 '$24.6 B',
 '$24.4 B',
 '$23.6 B',
 '$23.5 B',
 '$23.5 B',
 '$23.4 B',
 '$23.3 B',
 '$23 B',
 '$23 B',
 '$22.4 B',
 '$22 B',
 '$21.9 B',
 '$21.7 B',
 '$21.6 B',
 '$21.5 B',
 '$21.4 B',
 '$21.3 B',
 '$20.9 B',
 '$20.5 B',
 '

In [16]:
age_

['57',
 '49',
 '72',
 '65',
 '36',
 '90',
 '76',
 '48',
 '47',
 '63',
 '85',
 '67',
 '66',
 '65',
 '49',
 '81',
 '71',
 '72',
 '76',
 '79',
 '41',
 '50',
 '59',
 '58',
 '83',
 '56',
 '85',
 '58',
 '63',
 '56',
 '72',
 '84',
 '63',
 'N/A',
 '50',
 '75',
 '78',
 '81',
 '37',
 '56',
 '72',
 '70',
 '92',
 '55',
 '49',
 '63',
 '93',
 '81',
 '85',
 '39',
 '55',
 '52',
 '62',
 '58',
 '60',
 '76',
 '65',
 '83',
 '64',
 '42',
 '67',
 '85',
 '75',
 '88',
 '76',
 '70',
 '65',
 '82',
 '67',
 '67',
 '90',
 '75',
 '49',
 '78',
 '51',
 '50',
 '47',
 '68',
 '74',
 '93',
 '54',
 '70',
 '53',
 '73',
 '59',
 '80',
 '59',
 '71',
 '81',
 '57',
 '60',
 '54',
 '92',
 '50',
 '57',
 '65',
 '62',
 '70',
 '67',
 '43',
 '57',
 '54',
 '81',
 '74',
 '55',
 '49',
 '36',
 '39',
 '91',
 '56',
 '91',
 '36',
 '68',
 '81',
 '85',
 '66',
 '66',
 '55',
 '64',
 '52',
 '45',
 '63',
 '62',
 '85',
 '55',
 '82',
 '34',
 '56',
 '57',
 '50',
 '51',
 '88',
 'N/A',
 '70',
 '56',
 '51',
 '66',
 '91',
 '52',
 '91',
 '39',
 '54',
 '63

In [17]:
citizenship_

['United States',
 'United States',
 'France',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'United States',
 'India',
 'Spain',
 'France',
 'China',
 'United States',
 'China',
 'Mexico',
 'United States',
 'United States',
 'United States',
 'United States',
 'China',
 'United States',
 'United States',
 'India',
 'United States',
 'China',
 'United States',
 'United States',
 'Japan',
 'United States',
 'Japan',
 'France',
 'Canada',
 'Germany',
 'China',
 'United States',
 'China',
 'Germany',
 'China',
 'Italy',
 'France',
 'France',
 'Hong Kong',
 'China',
 'China',
 'United States',
 'Hong Kong',
 'United States',
 'United States',
 'China',
 'Russia',
 'Hong Kong',
 'China',
 'Germany',
 'Russia',
 'Austria',
 'China',
 'Germany',
 'Russia',
 'China',
 'Mexico',
 'Italy',
 'Japan',
 'United States',
 'United States',
 'Russia',
 'Russia',
 'United States',
 'China',
 'Australia',
 'United States',
 'India',
 'China',
 'Chile',
 'Chi

In [18]:
source_

['Amazon',
 'Tesla, SpaceX',
 'LVMH',
 'Microsoft',
 'Facebook',
 'Berkshire Hathaway',
 'software',
 'Google',
 'Google',
 'diversified',
 'Zara',
 "L'Oréal",
 'beverages, pharmaceuticals',
 'Microsoft',
 'internet media',
 'telecom',
 'Walmart',
 'Walmart',
 'Walmart',
 'Bloomberg LP',
 'e-commerce',
 'Amazon',
 'Quicken Loans',
 'infrastructure, commodities',
 'Nike',
 'e-commerce',
 'Koch Industries',
 'Koch Industries',
 'internet, telecom',
 'Dell computers',
 'fashion retail',
 'luxury goods',
 'media',
 'supermarkets',
 'package delivery',
 'casinos',
 'home appliances',
 'retail',
 'TikTok',
 'Nutella, chocolates',
 'Chanel',
 'Chanel',
 'diversified',
 'pig breeding',
 'online games',
 'music, chemicals',
 'real estate',
 'candy, pet food',
 'candy, pet food',
 'real estate',
 'steel, investments',
 'batteries',
 'real estate',
 'BMW, pharmaceuticals',
 'metals',
 'Red Bull',
 'soy sauce',
 'shipping',
 'steel, transport',
 'e-commerce',
 'mining',
 'eyeglasses',
 'sensors',


In [19]:
industry_

['Technology',
 'Automotive',
 'Fashion & Retail',
 'Technology',
 'Technology',
 'Finance & Investments',
 'Technology',
 'Technology',
 'Technology',
 'Diversified',
 'Fashion & Retail',
 'Fashion & Retail',
 'Food & Beverage',
 'Technology',
 'Technology',
 'Telecom',
 'Fashion & Retail',
 'Fashion & Retail',
 'Fashion & Retail',
 'Media & Entertainment',
 'Technology',
 'Technology',
 'Finance & Investments',
 'Diversified',
 'Fashion & Retail',
 'Technology',
 'Diversified',
 'Diversified',
 'Telecom',
 'Technology',
 'Fashion & Retail',
 'Fashion & Retail',
 'Media & Entertainment',
 'Fashion & Retail',
 'Service',
 'Gambling & Casinos',
 'Manufacturing',
 'Fashion & Retail',
 'Technology',
 'Food & Beverage',
 'Fashion & Retail',
 'Fashion & Retail',
 'Diversified',
 'Food & Beverage',
 'Technology',
 'Diversified',
 'Real Estate',
 'Food & Beverage',
 'Food & Beverage',
 'Real Estate',
 'Metals & Mining',
 'Energy',
 'Real Estate',
 'Automotive',
 'Metals & Mining',
 'Food & Be

In [10]:
print(len(name_),len(rank_),len(networth_), len(age_),len(citizenship_),len(source_),len(industry_))

3000 3000 3000 3000 3000 3000 3000


In [11]:
TOP_billionaires = pd.DataFrame({})
TOP_billionaires['Rank'] = rank_
TOP_billionaires['Name'] = name_
TOP_billionaires['Net Worth'] = networth_
TOP_billionaires['Age'] = age_
TOP_billionaires['Citizenship']  = citizenship_
TOP_billionaires['Source'] =  source_
TOP_billionaires['Industry'] = industry_
TOP_billionaires.head()

,Rank,Name,Net Worth,Age,Citizenship,Source,Industry
0,1,Jeff Bezos,$177 B,57,United States,Amazon,Technology
1,2,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
2,3,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
3,4,Bill Gates,$124 B,65,United States,Microsoft,Technology
4,5,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology


In [12]:
TOP_billionaires.to_csv("TOP_billionaires.csv")

# 9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [2]:
driver=webdriver.Chrome("chromedriver.exe")

In [3]:
driver.get('https://www.youtube.com')
time.sleep(2)
driver.find_element_by_id('search').send_keys('Hollywood is bleeding')
driver.find_element_by_id('search-icon-legacy').click()
time.sleep(3)
driver.find_element_by_xpath('//a[@class="yt-simple-endpoint style-scope ytd-video-renderer"]').click()
time.sleep(4)
driver.find_element_by_xpath('//button[@class="ytp-play-button ytp-button"]').click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=93.0.4577.82)


In [ ]:
for _ in range(0,500):
    driver.execute_script('window.scrollBy(0,5000)')
    time.sleep(3)
    driver.execute_script('window.scrollBy(0,-2000)')
    time.sleep(1)

# 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [4]:
driver=webdriver.Chrome("chromedriver.exe")

In [5]:
driver.get('https://www.hostelworld.com/')
time.sleep(4)
driver.find_element_by_id('location-text-input-field').click()
time.sleep(2)
driver.find_element_by_xpath('//*[@id="search-input-field"]').send_keys('London')
time.sleep(4)
driver.find_element_by_xpath('//*[@id="predicted-search-results"]/li[2]').click()
driver.find_element_by_id('search-button').click()

In [6]:
prifromprice = []
dormsfromprice = []
discitycentre = [] 


In [10]:
for l in range(0,3):
    try:
        dist_ = driver.find_elements_by_xpath('//div[@class="subtitle body-3"]//a//span')
        priv_ = driver.find_elements_by_xpath('//div[@class="prices-col"]//a//div[1]//p')
        dorm_ = driver.find_elements_by_xpath('//div[@class="prices-col"]//a[1]//div[2]')

        for i in dist_:
            discitycentre.append(i.text)
        
        for j in priv_:
            prifromprice.append(j.text.replace('Privates From\n',''))
        
        for k in dorm_:
            dormsfromprice.append(k.text.replace('Dorms From\n',''))
        
        driver.find_element_by_xpath('//div[@class="pagination-item pagination-next"]').click()
        time.sleep(4)
    except NoSuchElementException:
        discitycentre.append('-')
        prifromprice.append('-')
        dormsfromprice.append('-')

In [15]:
distcity = []
 
for j in range(0,len(discitycentre),2):
    distcity.append(discitycentre[j])
print(dormsfromprice)

['Rs1426', 'Rs1254', 'Rs2225', 'Rs1873', 'Rs2393', 'Rs1254', 'Rs1551', 'Rs1340', 'Rs1517', 'Rs1820', 'Rs1820', 'Rs1685', 'Rs3170', 'Rs1540', 'Rs1618', 'Rs1210', 'Rs2275', 'Rs1422', 'Rs1534', 'Rs1652', 'Rs1416', 'No Dorms Available', 'Rs1633', 'Rs1517', 'Rs1206', 'No Dorms Available', 'Rs2063', 'Rs2022', 'Rs1289', 'Rs2855', 'Rs1685', 'Rs1461', 'No Dorms Available', 'Rs1461', 'Rs2402', 'No Dorms Available', 'Rs2639', 'Rs1381', 'Rs1259', 'No Dorms Available', 'Rs753', 'Rs2528', 'Rs1618', 'Rs1011', 'Rs753', 'Rs2002', 'Rs3775', 'Rs2708', 'Rs1854', 'Rs1127', 'Rs1719', 'Rs1112', 'Rs1092', 'No Dorms Available', 'No Dorms Available', 'No Dorms Available', 'No Dorms Available', 'Rs1719', 'No Dorms Available', 'No Dorms Available', 'No Dorms Available', 'No Dorms Available', 'No Dorms Available', 'No Dorms Available', 'No Dorms Available', 'No Dorms Available', 'No Dorms Available', 'No Dorms Available', 'No Dorms Available', 'No Dorms Available', 'No Dorms Available', 'No Dorms Available', 'No D

In [16]:
print(len(prifromprice), len(distcity), len(dormsfromprice))

83 113 83


In [17]:
try:
    driver.find_element_by_xpath('//div[@class="pagination-item"]').click()
    time.sleep(2)
except:
    pass

urls = []
try:
    for _ in range(0,3):
        links = driver.find_elements_by_xpath('//div[@class="title-row"]//a')

        for url in links:
            urls.append(url.get_attribute('href'))
        driver.find_element_by_xpath('//div[@class="pagination-item pagination-next"]').click()
        time.sleep(5)
    len(urls)

except:
    pass

In [18]:
url_links = []
for i in range(0,len(urls),2):
    url_links.append(urls[i])
url_links


['https://www.hostelworld.com/pwa/hosteldetails.php/London-Waterloo-Hostel/London/39493?from=2021-09-28&to=2021-10-01&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/St-Christopher-s-Inn-London-Bridge/London/503?from=2021-09-28&to=2021-10-01&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/Barmy-Badger-Backpackers/London/64746?from=2021-09-28&to=2021-10-01&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/Wombat-s-The-City-Hostel-London/London/88047?from=2021-09-28&to=2021-10-01&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/Prime-Backpackers-Angel/London/93902?from=2021-09-28&to=2021-10-01&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/St-Christopher-s-Village/London/502?from=2021-09-28&to=2021-10-01&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/PubLove-The-Steam-Engine-Waterloo/London/29389?from=2021-09-28&to=2021-10-01&guests=2',
 'https://www.hostelworld.com/pwa/hosteldetails.php/Smart-Camden

In [20]:
hostelname = []
Ratings = []
totalreviews = []
overallreviews = []
Facilities = []
Description = []

for i in url_links:
    driver.get(i)
    time.sleep(3)
    try:
        name = driver.find_element_by_xpath('//div[@class="headline-container"]//div//h1//div')
        rate = driver.find_element_by_xpath('//div[@class="rating rating-summary-container big clickable"]//div')
        revs = driver.find_element_by_xpath('//div[@class="summary orange"]//div[2]')
        over = driver.find_element_by_xpath('//div[@class="summary orange"]//span')
        faci = driver.find_element_by_xpath('/html/body/div[1]/div/div/div[1]/section/div[11]/div/ul/li[1]/ul')
        desc = driver.find_element_by_xpath('//*[@id="__layout"]/div/div[1]/section/div[6]/div/div[2]/div[2]/div/div[2]')
        
        hostelname.append(name.text)
        Ratings.append(rate.text)
        totalreviews.append(revs.text)
        overallreviews.append(over.text)
        Facilities.append(faci.text)
        Description.append(desc.text)
        
    except NoSuchElementException:
        hostelname.append('-')
        Ratings.append('-')
        totalreviews.append('-')
        overallreviews.append('-')
        Facilities.append('-')
        Description.append('-')
        time.sleep(3)

In [21]:
hostelname

['-',
 "St Christopher's Inn - London Bridge",
 '-',
 '-',
 '-',
 "St Christopher's Village",
 '-',
 '-',
 '-',
 'Safestay London Elephant & Castle',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 "St Christopher's Oasis-Female Only",
 "St Christopher's Camden",
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 'Clink261',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-']

In [22]:
Ratings

['-',
 '7.0',
 '-',
 '-',
 '-',
 '8.6',
 '-',
 '-',
 '-',
 '8.0',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '8.1',
 '9.0',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '7.6',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-']

In [23]:
totalreviews

['-',
 '3237 Total Reviews',
 '-',
 '-',
 '-',
 '10855 Total Reviews',
 '-',
 '-',
 '-',
 '3958 Total Reviews',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '598 Total Reviews',
 '3709 Total Reviews',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '3178 Total Reviews',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-']

In [24]:
overallreviews

['-',
 'Very Good',
 '-',
 '-',
 '-',
 'Fabulous',
 '-',
 '-',
 '-',
 'Fabulous',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 'Fabulous',
 'Superb',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 'Very Good',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-']

In [25]:
Facilities

['-',
 'Linen Included Free City Maps Free WiFi Free Internet Access',
 '-',
 '-',
 '-',
 'Linen Included Free City Maps Free WiFi Free Internet Access Free City Tour',
 '-',
 '-',
 '-',
 'Linen Included Free City Maps Free WiFi',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 'Linen Included Free City Maps Towels Included Free WiFi Free Internet Access Free City Tour',
 'Linen Included Free City Maps Free WiFi Free Internet Access',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 'Linen Included Free City Maps Free WiFi',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-']

In [26]:
Description

['-',
 'COVID 19 Policy Update.\nIn response to Coronavirus (COVID-19), additional safety and sanitation measures are currently in effect at this property, including social distancing and in-house disinfection. \n\nBase yourself in the electric area of London Bridge at our flagship hostel St Christopher’s at The Inn. Here, you’ll get the best of both worlds with modern bath and bedrooms whilst having a traditional but lively 16th century English pub right downstairs with live music every day. Enjoy your morning breakfast in amongst the charm of The Inn. \n\nWithin just a short walk of our hostel you can reach the London Eye, Tower Bridge and London’s oldest and most famous food market, Borough Market. Getting about by tube couldn’t be easier as we’re located 2 minutes walk from both London Bridge AND Borough tube stations. \n\n*Please note that we require an in date government approved - drivers license, passport or national ID card at check-in.\n\n**Guests under the age of 18 must sta